In [ ]:
import caffe

In [ ]:
def do_train(solver, cl_args):
    tr_its_per_epoch = int(float(num_tr) / tr_batch_size)
    #val_its_per_epoch = int(float(num_val) / val_batch_size)
    num_epochs = cl_args["num_epochs"]

    loss_keys = ["L_cls","L_obj","L_xy","L_wh","L_rec","final_loss"]
    losses = {}
    for ep in range(num_epochs):
        losses[ep] = {k:[] for k in loss_keys}
        losses[str(ep) + "_mean"] = {k:0 for k in loss_keys}
        for it in range(tr_its_per_epoch):
            solver.step(1)

            for k in loss_keys:
                loss = np.float32(solver.net.blobs[k].data)
                losses[ep][k].append(loss)

        losses[str(ep) + "_mean"] = {k:np.mean(losses[ep][k]) for k in loss_keys}
        for k in loss_keys:
            loss = losses[str(ep) + "_mean"][k]
            sys.stderr.write("\n Epoch %i: Final %s Loss = %6.3f\n" % (ep, k, loss))
        suffix = "learning_curve_lr_%8.6f_fs_%4.3f_%s.jpg"%(cl_args["lr"], cl_args["filters_scale"], cl_args["data_dir"])
        plt.figure(1)
        plt.plot([losses[str(epoch) + "_mean"]["final_loss"] for epoch in range(ep + 1)])
        #plt.show()
        plt.savefig(join(cl_args["save_dir"],"tr_" + suffix))

In [ ]:
if __name__ == "__main__":
    def get_cl_args():
        cl_args = {"lr": 0.00001,
                   "num_epochs": 20, 
                   "filters_scale": 1./8,
                   "data_dir": "extremely_small_dataset", 
                   "save_dir": "/global/homes/r/racah/projects/climate-caffe/2d_semi_sup/notebooks/plots",
                   "tr_batch_size":4,
                   "val_batch_size":16 }

        if any(["jupyter" in arg for arg in sys.argv]):
            sys.argv=sys.argv[:1]

        parser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter)
        for k,v in cl_args.iteritems():
            parser.add_argument('--' + k, type=type(v), default=v, help=k)

        args = parser.parse_args()
        cl_args.update(args.__dict__)
        return cl_args
    
    
    #get solver
    solver_filename = ...
    num_tr = ...
    solver = caffe.SGDSolver(solver_filename)
    do_train(solver, cl_args)
    
    